## Nets in the Toplogical order

In [31]:
import networkx as nx
g=nx.DiGraph()
filename=input("Enter the netlist file:")
l=[]
gate={}
with open(filename) as f:
    lines=f.readlines()
    for line in lines:
        p=line.split()
        if p[1]!="inv" and p[1]!="buf":
            gate.update({p[4]:p[1]})
            l.append((p[2],p[4]))
            l.append((p[3],p[4]))
        elif p[1]=="inv" or p[1]=="buf":
            gate.update({p[3]:p[1]})
            l.append((p[2],p[3]))

g.add_edges_from(l)
nx.set_node_attributes(g,gate,name="gateType")
a=g.nodes(data=True)
for item in a:
    if not item[1].get('gateType'):
        item[1]['gateType'] = 'PI'
# print(a)
nl = list(nx.topological_sort(g))
print('Nodes in topological order',nl)

Enter the netlist file: c17.net


Nodes in topological order ['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23']


## Topologically ordered evaluation

In [32]:
def topo_evaluation(input_vectors,net_states,g):
    for iv in input_vectors:
        for i, node in enumerate(order):
            net_states[node] = iv[i]

        for node in nl:
            if g.nodes[node]['gateType'] == "PI" or net_states[node] is not None:
                continue
            input_nodes = list(g.predecessors(node))
            inputs=[net_states[n] for n in input_nodes]

            gate_type = g.nodes[node]['gateType']
            if gate_type == 'and2':
                net_states[node] = int(all(inputs))
            elif gate_type == 'nand2':
                net_states[node] = int(not all(inputs))
            elif gate_type == 'or2':
                net_states[node] = int(any(inputs))
            elif gate_type == 'nor2':
                net_states[node] = int(not any(inputs))
            elif gate_type == 'xor2':
                net_states[node] = int(inputs.count(1) % 2 == 1)
            elif gate_type == 'xnor2':
                net_states[node] = int(inputs.count(1) % 2 == 0)
            elif gate_type == 'buf':
                net_states[node] = int(inputs[0])
            elif gate_type == 'inv':
                net_states[node] = not int(inputs[0])
    return net_states

input_file=input("Enter the inputs file:")
with open(input_file) as i:
    lines=i.readlines()
    m=len(lines)-1
    order=lines[0].split()
    
for node in sorted(g.nodes()):
    print(f'{node}', end="  ")
print("\n")    
for j in range (1,m+1):
    print(f"#{j}")
    input_vectors = [list(map(int, lines[j].split()))]
    net_states ={node: None for node in g.nodes()}
    net_states=topo_evaluation(input_vectors,net_states,g)
    for node in sorted(net_states.keys()):
        state = net_states[node]
        l=len(node)
        print(f'{state:<{l}}', end="  ")
    print("\n")

Enter the inputs file: c17.inputs


N1  N2  N22  N23  N3  N6  N7  n_0  n_1  n_2  n_3  

#1
0   1   1    1    0   0   0   1    1    1    0    

#2
0   0   0    0    1   0   0   1    1    1    1    

#3
1   0   0    0    0   0   0   1    1    1    1    

#4
0   0   0    0    1   1   1   1    0    1    1    

#5
1   1   1    0    1   1   1   0    0    1    1    

#6
1   1   1    1    1   0   0   0    1    1    0    

#7
1   1   1    0    1   1   0   0    0    1    1    

#8
1   1   1    1    0   0   0   1    1    1    0    

#9
0   1   1    1    1   0   1   1    1    0    0    

#10
0   0   0    0    1   1   0   1    0    1    1    



### Explaination
Once the nets are arranged in the topological order, the state output of each gate is found based on the state of the inputs and the functionality of the logic gate.

In [33]:
%timeit topo_evaluation(input_vectors,net_states,g)

2.85 µs ± 91.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Event Driven Evalution

In [34]:
def event_driven(queue,net_states,g):
    i=0
    while len(queue)!= 0:
        if net_states[queue[0]] is not None:
            queue.pop(0)
            continue
        input_nodes = list(g.predecessors(queue[0]))
        inputs=[net_states[n] for n in input_nodes]
        check=0
        i+=1
        for item in inputs:
            if item is None:
                node=queue.pop(0)
                queue.append(node)
                check=1
                break
        if check ==1:
            continue
        else:
            gate_type = g.nodes[queue[0]]['gateType']
            if gate_type == 'and2':
                net_states[queue[0]] = int(all(inputs))
            elif gate_type == 'nand2':
                net_states[queue[0]] = int(not all(inputs))
            elif gate_type == 'or2':
                net_states[queue[0]] = int(any(inputs))
            elif gate_type == 'nor2':
                net_states[queue[0]] = int(not any(inputs))
            elif gate_type == 'xor2':
                net_states[queue[0]] = int(inputs.count(1) % 2 == 1)
            elif gate_type == 'xnor2':
                net_states[queue[0]] = int(inputs.count(1) % 2 == 0)
            elif gate_type == 'buf':
                net_states[queue[0]] = int(inputs[0])
            elif gate_type == 'inv':
                net_states[queue[0]] = not int(inputs[0])
            queue.pop(0)    
    return queue,net_states

queue=[]
input_file=input("Enter the inputs file:")
with open(input_file) as i:
    lines=i.readlines()
    m=len(lines)-1
    order=lines[0].split()
    n=len(order)
    input_vectors=list(map(int, lines[1].split()))

for node in g.nodes():
    queue.append(node)
net_states ={node: None for node in g.nodes()}
i=0
j=0
while i < n:
    if queue[j]== order[i]:
        net_states[queue[j]] = input_vectors[i]
        i+=1
        queue.remove(queue[j])
        j=0
    else:
        j+=1
for node in sorted(g.nodes()):
    print(f'{node}', end="  ")
print("\n")  
print("#1")
queue,net_states=event_driven(queue,net_states,g)
for node in sorted(net_states.keys()):
    state = net_states[node]
    l=len(node)
    print(f'{state:<{l}}', end="  ")
print("\n")

for j in range (2,m+1):
    print(f"#{j}")
    i=0
    input_vectors2 = list(map(int, lines[j].split()))
    change=[]
    for node in order:    
        if input_vectors2[i] != input_vectors[i]:
            queue.append(node)
            for n in queue:
                l = g.successors(n)
                queue.extend(l)
                queue = list(set(queue))
            i+=1
        else:
            i+=1
    queue = list(set(queue))
    for node in queue:
        net_states [node]=None
    n=len(queue)
    b=len(order)
    for q in range(0,n) :
        for k in range (0,b):
            if queue[q]== order[k]:
                net_states[queue[q]] = input_vectors2[k]
    queue,net_states=event_driven(queue,net_states,g)
    for node in sorted(net_states.keys()):
        state = net_states[node]
        l=len(node)
        print(f'{state:<{l}}', end="  ")
    print("\n")
    input_vectors=input_vectors2

Enter the inputs file: c17.inputs


N1  N2  N22  N23  N3  N6  N7  n_0  n_1  n_2  n_3  

#1
0   1   1    1    0   0   0   1    1    1    0    

#2
0   0   0    0    1   0   0   1    1    1    1    

#3
1   0   0    0    0   0   0   1    1    1    1    

#4
0   0   0    0    1   1   1   1    0    1    1    

#5
1   1   1    0    1   1   1   0    0    1    1    

#6
1   1   1    1    1   0   0   0    1    1    0    

#7
1   1   1    1    1   1   0   0    0    1    1    

#8
1   1   1    1    0   0   0   1    1    1    0    

#9
0   1   1    1    1   0   1   1    1    0    0    

#10
0   0   0    0    1   1   0   1    0    1    1    



### Explaination
we start with a net_states that contains the current state of each net, initialized to ‘None’ at the start of the simulation. We form a queue that tracks the nets whose state has to be re-evaluated.First, the states of the primary inputs needs to be updated so we add these to the queue. Next, we dequeue nets from the queue and evaluate the new state. If the new state is different from the current state, the sucessors of the net are added to the queue for evaluation.

In [35]:
%timeit event_driven(queue,net_states,g)

173 ns ± 0.662 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Which approach is faster/more efficient

Generally, we can say that the **Event Driven Approach** is more **faster and efficient** than **multiple rounds of evaluation using Topological sort**.This is because the Event driven approach makes the changes only for the **nets** who got affected by the change in inputs compared to the inputs given before. But in the other method, it followed updating even though the nets who didn't got effected since there predecessors weren't got changed. This making it to take more time than the Event driven approach.

But if the input of each and every net was changed in the cycles, then we can see the **Event driven approach** takes more time than the other one. this is because since almost every input got changed, thus creating a queue will consume a more time than usual. So, this rapid change in inputs (*may not be each and every but most of them*), our multiple rounds evaluation using Topological order will work finer than Event driven.